# Capstone Project 2

## Predict who will win an NFL bet

Data Source:  
    
    Main/Historical https://datasetsearch.research.google.com/search?query=NFL%20scores%20and%20betting%20data&docid=42gvYWceSRPsddH5AAAAAA%3D%3D

    Current Week Odds:     https://www.vegasinsider.com/nfl/odds/las-vegas                                              
    Current Year Results:  https://www.teamrankings.com/nfl-odds-week-#  (where # is week number)
    
    

In [1]:
# packages
import os
import pandas as pd
import numpy  as np
import csv
from datetime import datetime
import warnings


#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore",category=DeprecationWarning)
#    import sklearn
# required machine learning packages
#from sklearn import model_selection
#from sklearn.feature_selection import RFE
#from sklearn.metrics import brier_score_loss, roc_auc_score
#from sklearn.naive_bayes import GaussianNB
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#from sklearn.linear_model import LogisticRegression
#from sklearn.calibration import CalibratedClassifierCV as CCV
#from sklearn.neural_network import MLPClassifier
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
#import xgboost as xgb

In [2]:
datapath = "input/"

teams      = pd.read_csv(datapath + "nfl_teams.csv")
scores     = pd.read_csv(datapath + "nfl_scores.csv")
stadiums   = pd.read_csv(datapath + "nfl_stadiums.csv", encoding='latin-1')
scoreswork = pd.read_csv(datapath + "nfl_scores_2020work.csv", encoding='latin-1', index_col=False, header=None, 
                         names=['col1','col2','col3','col4','col5'])

In [3]:
scoreswork.info()
scoreswork.head(14)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1248 entries, 0 to 1247
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col1    930 non-null    object
 1   col2    624 non-null    object
 2   col3    624 non-null    object
 3   col4    624 non-null    object
 4   col5    624 non-null    object
dtypes: object(5)
memory usage: 48.9+ KB


,col1,col2,col3,col4,col5
0,2020 NFL Week 1 Odds,NaN,NaN,NaN,NaN
1,Latest OddsOdds History,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,Division:,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,Team: ...,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,"Thursday, September 10th, 2020",NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [4]:
# Assign team_id
def f_team_id(team_name):
    if team_name == 'Arizona':
        team_id = 'ARI'
    elif team_name == 'Baltimore':
        team_id = 'BAL'
    elif team_name == 'Buffalo':
        team_id = 'BUF'
    elif team_name == 'Carolina':
        team_id = 'CAR'
    elif team_name == 'Chicago':
        team_id = 'CHI'
    elif team_name == 'Cincinnati':
        team_id = 'CIN'
    elif team_name == 'Cleveland':
        team_id = 'CLE'
    elif team_name == 'Dallas':
        team_id = 'DAL'
    elif team_name == 'Denver':
        team_id = 'DEN'
    elif team_name == 'Detroit':
        team_id = 'DET'
    elif team_name == 'Green Bay':
        team_id = 'GB'
    elif team_name == 'Houston':
        team_id = 'HOU'
    elif team_name == 'Indianapolis':
        team_id = 'IND'
    elif team_name == 'Jacksonville':
        team_id = 'JAX'
    elif team_name == 'Kansas City':
        team_id = 'KC'
    elif team_name == 'LA Chargers':
        team_id = 'LAC'
    elif team_name == 'LA Rams':
        team_id = 'LAR'
    elif team_name == 'Las Vegas':
        team_id = 'LVR'
    elif team_name == 'Miami':
        team_id = 'MIA'
    elif team_name == 'Minnesota':
        team_id = 'MIN'
    elif team_name == 'New England':
        team_id = 'NE'
    elif team_name == 'New Orleans':
        team_id = 'NO'
    elif team_name == 'NY Giants':
        team_id = 'NYG'
    elif team_name == 'NY Jets':
        team_id = 'NYJ'
    elif team_name == 'Philadelphia':
        team_id = 'PHI'
    elif team_name == 'Pittsburgh':
        team_id = 'PIT'
    elif team_name == 'San Francisco':
        team_id = 'SF'
    elif team_name == 'Seattle':
        team_id = 'SEA'
    elif team_name == 'St. Louis':
        team_id = 'SCA'
    elif team_name == 'Tampa Bay':
        team_id = 'TB'
    elif team_name == 'Tennessee':
        team_id = 'TEN'
    elif team_name == 'Washington':
        team_id = 'WFT'
    else:
        team_id = 'ZZZ'
    
    return team_id

In [5]:
# Convert current year information
scoreswork['col1'] = scoreswork['col1'].astype(str)
team_away = 'x'
team_home = 'x'
schedule_playoff = 'FALSE'

nfl_scores_current = pd.DataFrame(columns=
                     ['schedule_date', 
                      'schedule_season', 
                      'schedule_week', 
                      'schedule_playoff',
                      'team_home',
                      'score_home',
                      'score_away',
                      'team_away',
                      'team_favorite_id',
                      'spread_favorite',
                      'over_under_line'])

for label, row in scoreswork.iterrows():

#   if scoreswork['col1'][ind].contains('NFL Week',regex=False):
    if row['col1'].find('NFL Week') > -1:
        schedule_season = row['col1'][0:4]
        schedule_week = row['col1'][14:16]
    elif row['col1'].find('st, 2020') > -1:
        schedule_date = datetime.strptime(row['col1'], "%A, %B %dst, %Y").date()
    elif row['col1'].find('nd, 2020') > -1:
        schedule_date = datetime.strptime(row['col1'], "%A, %B %dnd, %Y").date()
    elif row['col1'].find('rd, 2020') > -1:
        schedule_date = datetime.strptime(row['col1'], "%A, %B %drd, %Y").date()
    elif row['col1'].find('th, 2020') > -1:
        schedule_date = datetime.strptime(row['col1'], "%A, %B %dth, %Y").date()
    elif row['col1'].find(', 2020') > -1:
        schedule_date = datetime.strptime(row['col1'], "%A, %B %d, %Y").date()
    elif row['col1'].find('EST') > -1:
        team_away = ''
        team_home = ''
        spread_favorite = 0
        team_favorite_id = ''
    elif team_away == '':
        team_away=row['col1']
        score_away=row['col2']
        if float(row['col3']) < 0:
            spread_favorite = float(row['col3'])
            team_favorite_id = f_team_id(team_away)
        if row['col4'] not in ['over','under','push']:
            over_under_line = float(row['col4'])
    elif team_home == '':
        team_home=row['col1']
        score_home=row['col2']
        if float(row['col3']) < 0:
            spread_favorite = float(row['col3'])
            team_favorite_id = f_team_id(team_home)
        if row['col4'] not in ['over','under','push']:
            over_under_line = float(row['col4'])

        
        nfl_scores_data = [schedule_date, 
                      schedule_season, 
                      schedule_week, 
                      schedule_playoff,
                      team_home,
                      score_home,
                      score_away,
                      team_away,
                      team_favorite_id,
                      spread_favorite,
                      over_under_line]
        nfl_scores_current = pd.concat([pd.DataFrame([nfl_scores_data], columns=nfl_scores_current.columns), nfl_scores_current])

nfl_scores_current.reset_index()
nfl_scores_current.head()
nfl_scores_current.to_csv(datapath + "nfl_scores_2020.csv", index=False)

In [6]:
stadiums.info()
stadiums.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   stadium_name                  100 non-null    object 
 1   stadium_location              100 non-null    object 
 2   stadium_open                  76 non-null     float64
 3   stadium_close                 41 non-null     float64
 4   stadium_type                  93 non-null     object 
 5   stadium_address               91 non-null     object 
 6   stadium_weather_station_code  90 non-null     object 
 7   stadium_weather_type          93 non-null     object 
 8   stadium_capacity              39 non-null     object 
 9   stadium_surface               53 non-null     object 
 10  STATION                       52 non-null     object 
 11  NAME                          52 non-null     object 
 12  LATITUDE                      52 non-null     float64
 13  LONGIT

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_code,stadium_weather_type,stadium_capacity,stadium_surface,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION
0,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,dome,72000,FieldTurf,NaN,NaN,NaN,NaN,NaN
1,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alumni Stadium,"Chestnut Hill, MA",NaN,NaN,outdoor,"Perimeter Rd, Chestnut Hill, MA 02467",2467,cold,NaN,Grass,NaN,NaN,NaN,NaN,NaN
3,Anaheim Stadium,"Anaheim, CA",1980.0,1994.0,outdoor,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,warm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arrowhead Stadium,"Kansas City, MO",1972.0,NaN,outdoor,"1 Arrowhead Dr, Kansas City, MO 64129",64129,cold,"76,416",Grass,US1MOJC0028,"KANSAS CITY 5.1 SE, MO US",39.0692,-94.4871,264.9


In [7]:
teams = teams.drop(columns=['team_name_short', 'team_id_pfr','team_conference','team_division','team_conference_pre2002','team_division_pre2002'])
teams.info()
teams.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   team_name  43 non-null     object
 1   team_id    43 non-null     object
dtypes: object(2)
memory usage: 816.0+ bytes


,team_name,team_id
0,Arizona Cardinals,ARI
1,Atlanta Falcons,ATL
2,Baltimore Ravens,BAL
3,Boston Patriots,BOS
4,Buffalo Bills,BUF


In [8]:
# Only use data since 2010
scores = scores[scores['schedule_season'] >= 2010]
scores['over_under_line'] = pd.to_numeric(scores['over_under_line'])
scores = scores.drop(columns=['stadium', 'stadium_neutral', 'weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail'])
scores.info()
scores.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2670 entries, 10008 to 12677
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   schedule_date     2670 non-null   object 
 1   schedule_season   2670 non-null   int64  
 2   schedule_week     2670 non-null   object 
 3   schedule_playoff  2670 non-null   bool   
 4   team_home         2670 non-null   object 
 5   score_home        2670 non-null   int64  
 6   score_away        2670 non-null   int64  
 7   team_away         2670 non-null   object 
 8   team_favorite_id  2670 non-null   object 
 9   spread_favorite   2670 non-null   float64
 10  over_under_line   2670 non-null   float64
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 232.1+ KB


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line
10008,9/9/2010,2010,1,False,New Orleans Saints,14,9,Minnesota Vikings,NO,-5.0,49.5
10009,9/12/2010,2010,1,False,Buffalo Bills,10,15,Miami Dolphins,MIA,-3.0,39.0
10010,9/12/2010,2010,1,False,Chicago Bears,19,14,Detroit Lions,CHI,-6.5,45.0
10011,9/12/2010,2010,1,False,Houston Texans,34,24,Indianapolis Colts,IND,-1.0,48.0
10012,9/12/2010,2010,1,False,Jacksonville Jaguars,24,17,Denver Broncos,JAX,-3.0,41.5


In [9]:
scores_teams = pd.merge(scores,teams,left_on='team_favorite_id',right_on='team_id',how='left')
scores_teams.info()
scores_teams.tail(30)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2670 entries, 0 to 2669
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   schedule_date     2670 non-null   object 
 1   schedule_season   2670 non-null   int64  
 2   schedule_week     2670 non-null   object 
 3   schedule_playoff  2670 non-null   bool   
 4   team_home         2670 non-null   object 
 5   score_home        2670 non-null   int64  
 6   score_away        2670 non-null   int64  
 7   team_away         2670 non-null   object 
 8   team_favorite_id  2670 non-null   object 
 9   spread_favorite   2670 non-null   float64
 10  over_under_line   2670 non-null   float64
 11  team_name         2654 non-null   object 
 12  team_id           2654 non-null   object 
dtypes: bool(1), float64(2), int64(3), object(7)
memory usage: 273.8+ KB


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,team_name,team_id
2640,12/22/2019,2019,16,False,Tennessee Titans,28,38,New Orleans Saints,NO,-3.0,49.0,New Orleans Saints,NO
2641,12/22/2019,2019,16,False,Washington Redskins,35,41,New York Giants,NYG,-1.0,42.0,NY Giants,NYG
2642,12/23/2019,2019,16,False,Minnesota Vikings,10,23,Green Bay Packers,MIN,-4.0,46.5,Minnesota Vikings,MIN
2643,12/29/2019,2019,17,False,Baltimore Ravens,28,10,Pittsburgh Steelers,PIT,-1.0,35.0,Pittsburgh Steelers,PIT
2644,12/29/2019,2019,17,False,Buffalo Bills,6,13,New York Jets,NYJ,-1.0,37.0,NY Jets,NYJ
2645,12/29/2019,2019,17,False,Carolina Panthers,10,42,New Orleans Saints,NO,-14.0,44.5,New Orleans Saints,NO
2646,12/29/2019,2019,17,False,Cincinnati Bengals,33,23,Cleveland Browns,CLE,-2.5,43.5,Cleveland Browns,CLE
2647,12/29/2019,2019,17,False,Dallas Cowboys,47,16,Washington Redskins,DAL,-12.5,47.5,Dallas Cowboys,DAL
2648,12/29/2019,2019,17,False,Denver Broncos,16,15,Oakland Raiders,DEN,-4.5,41.0,Denver Broncos,DEN
2649,12/29/2019,2019,17,False,Detroit Lions,20,23,Green Bay Packers,GB,-13.0,44.0,Green Bay Packers,GB


In [10]:
# Find where team_favorite_id is not found
scores_teams[scores_teams['team_id'].isnull()]

# It turns out team_favorite_id is not found when it is 'PICK' and spread_favorite is 0.0

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,team_name,team_id
126,11/7/2010,2010,9,False,Oakland Raiders,23,20,Kansas City Chiefs,PICK,0.0,41.0,NaN,NaN
137,11/14/2010,2010,10,False,Jacksonville Jaguars,31,24,Houston Texans,PICK,0.0,48.5,NaN,NaN
194,12/12/2010,2010,14,False,Buffalo Bills,13,6,Cleveland Browns,PICK,0.0,39.0,NaN,NaN
211,12/19/2010,2010,15,False,Cincinnati Bengals,19,17,Cleveland Browns,PICK,0.0,41.0,NaN,NaN
252,1/2/2011,2010,17,False,Philadelphia Eagles,13,14,Dallas Cowboys,PICK,0.0,43.5,NaN,NaN
624,10/15/2012,2012,6,False,San Diego Chargers,24,35,Denver Broncos,PICK,0.0,47.5,NaN,NaN
744,12/16/2012,2012,15,False,Atlanta Falcons,34,0,New York Giants,PICK,0.0,50.0,NaN,NaN
892,10/14/2013,2013,6,False,San Diego Chargers,19,9,Indianapolis Colts,PICK,0.0,50.5,NaN,NaN
932,11/3/2013,2013,9,False,Washington Redskins,30,24,San Diego Chargers,PICK,0.0,48.5,NaN,NaN
1372,9/27/2015,2015,3,False,Dallas Cowboys,28,39,Atlanta Falcons,PICK,0.0,43.5,NaN,NaN


In [11]:
def f_spread (df):

    if df['team_home'] == df['team_name']:
        spread_home = df['spread_favorite']

    else:
        spread_home = df['spread_favorite'] * -1
    
    if   df['score_home'] + spread_home == df['score_away']:
           df['spread_home'] =  0
           df['spread_away'] =  0
    elif df['score_home'] + spread_home >  df['score_away']:
           df['spread_home'] =  1
           df['spread_away'] = -1
    else:
           df['spread_home'] = -1
           df['spread_away'] =  1
        
    return df

scores_teams = scores_teams.apply(f_spread, axis = 1)
        
scores_teams.tail(10)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,team_name,team_id,spread_home,spread_away
2660,1/4/2020,2019,Wildcard,True,New England Patriots,13,20,Tennessee Titans,NE,-4.5,45.0,New England Patriots,NE,-1,1
2661,1/5/2020,2019,Wildcard,True,New Orleans Saints,20,26,Minnesota Vikings,NO,-7.5,49.5,New Orleans Saints,NO,-1,1
2662,1/5/2020,2019,Wildcard,True,Philadelphia Eagles,9,17,Seattle Seahawks,PHI,-1.0,44.5,Philadelphia Eagles,PHI,-1,1
2663,1/11/2020,2019,Division,True,Baltimore Ravens,12,28,Tennessee Titans,BAL,-10.0,47.5,Baltimore Ravens,BAL,-1,1
2664,1/11/2020,2019,Division,True,San Francisco 49ers,27,10,Minnesota Vikings,SF,-7.0,44.5,San Francisco 49ers,SF,1,-1
2665,1/12/2020,2019,Division,True,Green Bay Packers,28,23,Seattle Seahawks,GB,-4.5,45.5,Green Bay Packers,GB,1,-1
2666,1/12/2020,2019,Division,True,Kansas City Chiefs,51,31,Houston Texans,KC,-10.0,50.5,Kansas City Chiefs,KC,1,-1
2667,1/19/2020,2019,Conference,True,Kansas City Chiefs,35,24,Tennessee Titans,KC,-7.0,51.0,Kansas City Chiefs,KC,1,-1
2668,1/19/2020,2019,Conference,True,San Francisco 49ers,37,20,Green Bay Packers,SF,-8.0,46.5,San Francisco 49ers,SF,1,-1
2669,2/2/2020,2019,Superbowl,True,Kansas City Chiefs,31,20,San Francisco 49ers,KC,-1.5,53.0,Kansas City Chiefs,KC,1,-1


In [12]:
def f_over_under (df):
   
    if   df['score_home'] + df['score_away'] == df['over_under_line']:
           df['over_under_winner'] =  0
    elif df['score_home'] + df['score_away'] >  df['over_under_line']:
           df['over_under_winner'] =  1
    else:
           df['over_under_winner'] = -1
        
    return df

scores_teams = scores_teams.apply(f_over_under, axis = 1)
        
scores_teams.tail(30)

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,team_name,team_id,spread_home,spread_away,over_under_winner
2640,12/22/2019,2019,16,False,Tennessee Titans,28,38,New Orleans Saints,NO,-3.0,49.0,New Orleans Saints,NO,-1,1,1
2641,12/22/2019,2019,16,False,Washington Redskins,35,41,New York Giants,NYG,-1.0,42.0,NY Giants,NYG,-1,1,1
2642,12/23/2019,2019,16,False,Minnesota Vikings,10,23,Green Bay Packers,MIN,-4.0,46.5,Minnesota Vikings,MIN,-1,1,-1
2643,12/29/2019,2019,17,False,Baltimore Ravens,28,10,Pittsburgh Steelers,PIT,-1.0,35.0,Pittsburgh Steelers,PIT,1,-1,1
2644,12/29/2019,2019,17,False,Buffalo Bills,6,13,New York Jets,NYJ,-1.0,37.0,NY Jets,NYJ,-1,1,-1
2645,12/29/2019,2019,17,False,Carolina Panthers,10,42,New Orleans Saints,NO,-14.0,44.5,New Orleans Saints,NO,-1,1,1
2646,12/29/2019,2019,17,False,Cincinnati Bengals,33,23,Cleveland Browns,CLE,-2.5,43.5,Cleveland Browns,CLE,1,-1,1
2647,12/29/2019,2019,17,False,Dallas Cowboys,47,16,Washington Redskins,DAL,-12.5,47.5,Dallas Cowboys,DAL,1,-1,1
2648,12/29/2019,2019,17,False,Denver Broncos,16,15,Oakland Raiders,DEN,-4.5,41.0,Denver Broncos,DEN,-1,1,-1
2649,12/29/2019,2019,17,False,Detroit Lions,20,23,Green Bay Packers,GB,-13.0,44.0,Green Bay Packers,GB,1,-1,-1


In [13]:
scores_teams.groupby(['team_home','schedule_season'])['spread_home'].mean()

team_home            schedule_season
Arizona Cardinals    2010              -0.250000
                     2011               0.125000
                     2012              -0.250000
                     2013               0.250000
                     2014               0.500000
                                          ...   
Washington Redskins  2015               0.111111
                     2016               0.000000
                     2017               0.000000
                     2018               0.000000
                     2019              -0.500000
Name: spread_home, Length: 320, dtype: float64

In [14]:
scores_teams.groupby(['team_away','schedule_season'])['spread_away'].mean()

team_away            schedule_season
Arizona Cardinals    2010              -0.500
                     2011               0.000
                     2012              -0.250
                     2013               0.125
                     2014               0.000
                                        ...  
Washington Redskins  2015               0.000
                     2016               0.250
                     2017              -0.250
                     2018               0.250
                     2019               0.000
Name: spread_away, Length: 320, dtype: float64

In [15]:
scores_teams.groupby(['team_home','schedule_season'])['over_under_winner'].mean()

team_home            schedule_season
Arizona Cardinals    2010               0.500000
                     2011               0.125000
                     2012               0.000000
                     2013               0.000000
                     2014              -0.250000
                                          ...   
Washington Redskins  2015              -0.111111
                     2016               0.500000
                     2017              -0.250000
                     2018               0.000000
                     2019               0.250000
Name: over_under_winner, Length: 320, dtype: float64

In [16]:
scores_teams.groupby(['team_away','schedule_season'])['over_under_winner'].mean()

team_away            schedule_season
Arizona Cardinals    2010               0.000000
                     2011              -0.500000
                     2012              -0.250000
                     2013               0.000000
                     2014              -0.222222
                                          ...   
Washington Redskins  2015               0.500000
                     2016               0.500000
                     2017               0.000000
                     2018              -0.250000
                     2019              -0.250000
Name: over_under_winner, Length: 320, dtype: float64